In [31]:
from models_config import build_analyzer
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import PatternRecognizer
from log_analysis import analyze_save

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [32]:
file_path = "data/sample.txt"
text = open(file_path).read()
print(text)

UNIVERSITY MEDICAL CTR
Department:  Neurology / Internal Med
Document Type:  ED NOTE + CONSULT + DISCHARGE
Generated: 2024.03.22  07:14 AM
Printed by: J.Nguyen (unit clerk)   ext 4021

----------------------------------------------------------------------
PATIENT INFORMATION
----------------------------------------------------------------------
Name:   Jenniferr   K.  Lee
AKA:    Jenny Lee / J. Lee
DOB:    5-6-94
Age:    29 y/o
Sex:    F
Preferred Language: English
Race/ethnicity (self-reported):  Korean-American

MRN:    00077219
Acct#:  A-77821
Encounter ID: 44-92-771

Home Address:
  455   San Mateo Ave
  Apt#  3B
  Redwood   City,  CA  94063

CURRENT ADDRESS (per patient, moved recently):
  4127 W. Elm st   Apt  #3B
  Springfeld, IL  62704

Phone:  312-555-7712
Alt/cell: (312) 555  77  13
Email:  jlee94 @ yahoo.com
Emergency contact:
  Marry  Smi th (spouse)  217.555.0198
  relationship: wife
Secondary contact:
  Anne McKinly (sister)  773.555.8891

Primary care provider (outside):

In [33]:
stanford = build_analyzer()[0]

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 142524.89it/s]
Device set to use cpu


In [34]:
# results = stanford.analyze(text=text, language="en")
first_pass, pass_idx = analyze_save(stanford, text, doc_id=1, case="sample")
print(first_pass)

<LOCATION>
Department:  Neurology / Internal Med
Document Type:  ED NOTE + CONSULT + DISCHARGE
Generated: <PHONE_NUMBER>:14 AM
Printed by: <PERSON> (unit <PERSON>)   ext 4021

----------------------------------------------------------------------
PATIENT INFORMATION
----------------------------------------------------------------------
Name:   <PERSON>
AKA:    <PERSON> / <PERSON>
DOB:    <DATE_TIME>
<AGE>
Sex:    F
Preferred Language: English
Race/ethnicity (self-reported):  Korean-American

<MRN>
Acct#:  <ID>
Encounter ID: <ID>

Home Address:
  <LOCATION>
  Apt#  3B
  <LOCATION>  <ZIPCODE>

CURRENT ADDRESS (per patient, moved recently):
  <LOCATION>   Apt  #3B
  <LOCATION>  <PHONE_NUMBER>

Phone:  <PHONE_NUMBER>
Alt/cell: <PHONE_NUMBER>
Email:  <ID> @ <ORGANIZATION>
Emergency contact:
  <PERSON> (spouse)  <PHONE_NUMBER>
  relationship: wife
Secondary contact:
  <PERSON> (sister)  <PHONE_NUMBER>

Primary care provider (outside):
  <PERSON>
  <LOCATION> Pulmonology
  Fax: <PHONE_NUMBER>

In [35]:
# anonymized_text = AnonymizerEngine().anonymize(text=text,analyzer_results=results)
# print(text, "\n------------\n")

# first_pass = anonymized_text.text


In [36]:
# Blacklist + Whitelist

deny_list = ["Pulmonology", "Neurology", "Internal Med"]
allow_list = ["2/28/24", "1990s"]

# deny_recognizer = PatternRecognizer(supported_entity="HITL", deny_list=deny_list)
# stanford.registry.add_recognizer(deny_recognizer)

second_pass, pass_idx = analyze_save(stanford, text, case="sample", doc_id=pass_idx, allow_list=allow_list, deny_list=deny_list)
# second_results = stanford.analyze(text=text, language="en", allow_list=allow_list)
# second_pass = AnonymizerEngine().anonymize(text=text,analyzer_results=second_results)
# print(second_pass.text)

In [37]:
print(second_pass)

<LOCATION>
Department:  <HITL> / <HITL>
Document Type:  ED NOTE + CONSULT + DISCHARGE
Generated: <PHONE_NUMBER>:14 AM
Printed by: <PERSON> (unit <PERSON>)   ext 4021

----------------------------------------------------------------------
PATIENT INFORMATION
----------------------------------------------------------------------
Name:   <PERSON>
AKA:    <PERSON> / <PERSON>
DOB:    <DATE_TIME>
<AGE>
Sex:    F
Preferred Language: English
Race/ethnicity (self-reported):  Korean-American

<MRN>
Acct#:  <ID>
Encounter ID: <ID>

Home Address:
  <LOCATION>
  Apt#  3B
  <LOCATION>  <ZIPCODE>

CURRENT ADDRESS (per patient, moved recently):
  <LOCATION>   Apt  #3B
  <LOCATION>  <PHONE_NUMBER>

Phone:  <PHONE_NUMBER>
Alt/cell: <PHONE_NUMBER>
Email:  <ID> @ <ORGANIZATION>
Emergency contact:
  <PERSON> (spouse)  <PHONE_NUMBER>
  relationship: wife
Secondary contact:
  <PERSON> (sister)  <PHONE_NUMBER>

Primary care provider (outside):
  <PERSON>
  <LOCATION> <HITL>
  Fax: <PHONE_NUMBER>

------------